<a href="https://colab.research.google.com/github/sagargahalod/Git-test/blob/branch1/CRF25_Recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import json
import pandas as pd

In [85]:
with open("CFR25_GroundTruth.json", encoding='utf-8', errors='ignore') as json_data:
    GT = json.load(json_data, strict=False)




In [86]:
with open("CFR25_diff.json", encoding='utf-8', errors='ignore') as json_data1:
    diff = json.load(json_data1, strict=False)



In [84]:
LCW_Total = GT_Total = 0

Add_tot = Del_tol = Mod_tot = Identical_tot = 0

Mean_list = []

In [166]:
l1 = [1,2,3,4,5]

print(sum(l1)/len(l1))

3.0


In [167]:
def Recall(LCW_Total,GT_Total):
  #calculation id level average
  Idrecall = LCW_Total / GT_Total
  print('The recall for id {0} is : {1}'.format(GT_Total.key,Idrecall))
  Mean_list.append(Idrecall)
  
  
  #calculation mean recall
  Meanrecall = sum(Mean_list) / len(Mean_list)
  print('The mean recall across the controls : {}'.format(Meanrecal))
  return Idrecall , Meanrecall

In [88]:
for gt in GT:
  if gt['id'] in ('Added','Deleted','Identical'):
    GT_Total =+1 
    #Recall(Tot)

In [194]:
GT_Total = 0
GT_id = {}
IDrecall_list = []
for gt in GT:

  if gt['type'] == 'Modified':  
    GT_Total +=1     
  if gt['IdChange'] == "true":
    GT_Total +=1  
  if gt['Cntrl_Grp_Change'] == 'true':
    GT_Total +=1    
  if gt['Title_Change'] == 'true':
    GT_Total +=1
    
  #counting the changes related to subpart level
   
  GT_Total = GT_Total + len(gt['sub_identical']) + len(gt['sub_added']) + len(gt['sub_removed']) + len(gt['sub_modified'])
  GT_id[gt['id']] = GT_Total 

  print('Actual changes as per the Groundtruth for ID {0} : {1}'.format(gt['id'],GT_Total)) 
   
  
  Tool_result = comparison(gt)
  print('Changes captured by the tool for ID {0} : {1}'.format(gt['id'],Tool_result))

  ID_recall = (Tool_result/GT_Total)*100
  
  print('Recall for this ID : ',ID_recall)
  print('\n')
  IDrecall_list.append(ID_recall)
  GT_Total = 0

Total_recall = sum(IDrecall_list)/len(IDrecall_list) 
print('Total reacall : ',Total_recall)

Actual changes as per the Groundtruth for ID 25.26 : 16
Changes captured by the tool for ID 25.26 : 15
Recall for this ID :  93.75


Actual changes as per the Groundtruth for ID 25.21 : 10
Changes captured by the tool for ID 25.21 : 8
Recall for this ID :  80.0


Actual changes as per the Groundtruth for ID 25.27 : 24
Changes captured by the tool for ID 25.27 : 13
Recall for this ID :  54.166666666666664


Actual changes as per the Groundtruth for ID 25.63 : 6
Changes captured by the tool for ID 25.63 : 6
Recall for this ID :  100.0


Actual changes as per the Groundtruth for ID 25.12 : 5
Changes captured by the tool for ID 25.12 : 2
Recall for this ID :  40.0


Actual changes as per the Groundtruth for ID 25.25 : 13
Changes captured by the tool for ID 25.25 : 8
Recall for this ID :  61.53846153846154


Actual changes as per the Groundtruth for ID 25.11 : 10
Changes captured by the tool for ID 25.11 : 10
Recall for this ID :  100.0


Actual changes as per the Groundtruth for ID 25.43 :

In [159]:
def comparison(gt):
  Id_flag = 0 
  LCW_Total = 0

  LCW_Total = LCW_Total + control(gt)
  LCW_Total = LCW_Total + cntrlgrp(gt)
  LCW_Total = LCW_Total + title(gt)
  LCW_Total = LCW_Total + content(gt)
  
  return(LCW_Total)
  

In [171]:
def content(gt):
  LCW_Total = 0          #Local variable

  for sub_id in gt['sub_identical']:
    for sub in diff:
      if (sub['old_id'] == sub_id) & (sub['node_type'] == 'content') & (sub['status'] == 'Identical'):
        LCW_Total += 1
  for sub_id in gt['sub_added']:
    for sub in diff:
      if (sub['new_id'] == sub_id) & (sub['node_type'] == 'content') & (sub['status'] == 'Added') :
        LCW_Total += 1
  for sub_id in gt['sub_removed']:
    for sub in diff:
      if (sub['old_id'] == sub_id) & (sub['node_type'] == 'content') & (sub['status'] == 'Removed') :
        LCW_Total += 1
  for sub_id in gt['sub_modified']:
    for sub in diff:
      if (sub['old_id'] == sub_id) & (sub['node_type'] == 'content') & (sub['status'] == 'Modified') :
        LCW_Total += 1

  return(LCW_Total)

In [170]:
def control(gt):
  LCW_Total = 0        #local variable
  Id_flag = 0
  for df in diff:  
    
    if (gt["id"] == df["old_id"]) & (df['node_type'] == 'control') & (Id_flag == 0):
      
      if gt['type'] == df['status']:
        LCW_Total +=1
        Id_flag = 1
      
      Id_flag = 0
      if (df['new_id'] != '') & (df['old_id'] != df['new_id']) &  (gt['IdChange'] =='true') & (Id_flag == 0):
        LCW_Total +=1
        Id_flag = 1

  return(LCW_Total)

    

In [169]:
def cntrlgrp(gt):
  LCW_Total = 0                 #Local variable  
  cntrl_grp = gt['id'] + '_cntlGrp'
  #print(cntrl_grp) 
  Id_flag = 0      
  for df in diff:   
        
    if (gt['Cntrl_Grp_Change'] == 'true') & (df['node_type'] == 'control_field') & (df['old_id'] == cntrl_grp) & (Id_flag == 0) : 
       
      if df['status'] == 'Modified':
        LCW_Total +=1
        Id_flag = 1
  return(LCW_Total) 

In [168]:
def title(gt):
  LCW_Total = 0                 #Local variable
  title = gt['id'] + '_title'
  #print(title)   
  Id_flag = 0    
  for df in diff:   
       
    if (gt['Title_Change'] == 'true') & (df['node_type'] == 'control_field') & (df['old_id'] == title) & (Id_flag == 0): 
       
      if df['status'] == 'Modified':
        LCW_Total +=1
        Id_flag = 1
  return(LCW_Total) 